In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-06-08 16:31:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2




<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
using Pkg

In [ ]:
Pkg.add(url="https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git")

In [ ]:
Pkg.add("QuantumControl")

In [ ]:
using GRAPELinesearchAnalysis

In [ ]:
using QuantumControl

In [ ]:
Pkg.add("QuantumControlBase")

In [ ]:
ϵ1(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);
ϵ2(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);
ϵ3(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);
ϵ4(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);
ϵ5(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);
ϵ6(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=700, t_rise=0.3, func=:blackman);

In [ ]:
Iσ̂_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0  1  0
        0  0  0  -1
    ]
    Iσ̂_x = ComplexF64[
        0  1  0  0
        1  0  0  0
        0  0  0  1
        0  0  1  0
    ]
    σ̂_zI = ComplexF64[
        1  0  0  0
        0  1  0  0
        0  0 -1  0
        0  0  0  -1
    ]
    σ̂_x_x = ComplexF64[
        0  0  0  1
        0  0  1  0
        0  1  0  0
        1  0  0  0
    ]
    σ̂_z_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0 -1  0
        0  0  0  1
    ]
    σ̂_y_y = ComplexF64[
        0  0  0 -1
        0  0  1  0
        0  1  0  0
       -1  0  0  0
    ]
    σ̂_z_z_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0 -1
    ]
    I_σ̂_z_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]
    σ̂_z_I_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]
     σ̂_z_I_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
    ]
    I_I_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0 -1
    ]
    I_σ̂_z_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
    ]
    I_I_σ̂_x = ComplexF64[
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
    ]
    I_σ̂_xI = ComplexF64[
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
    ]
    σ̂_xI_I = ComplexF64[
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
    ]

    σ̂_x_σ̂_x_I = ComplexF64[
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
    ]
    σ̂_y_σ̂_y_I = ComplexF64[
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
       -1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
    ]
    σ̂_z_σ̂_z_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
    ]
    σ̂_xI_σ̂_x = ComplexF64[
        0  0  0  0  0  1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  1  0  0  0  0  0
    ]
    σ̂_yI_σ̂_y = ComplexF64[
        0  0  0  0  0 -1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0 -1
        0  0  0  0  0  0  1  0
        0  1  0  0  0  0  0  0
       -1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0 -1  0  0  0  0  0
    ]
    σ̂_zI_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]

    I_I_σ̂_y = ComplexF64[
        0  -1im  0  0  0  0  0  0
        1im  0  0  0  0  0  0  0
        0  0  0  -1im  0  0  0  0
        0  0  1im  0  0  0  0  0
        0  0  0  0  0  -1im  0  0
        0  0  0  0  1im  0  0  0
        0  0  0  0  0  0  0  -1im
        0  0  0  0  0  0  1im  0
    ]
    I_σ̂_y_I = ComplexF64[
        0  0  -1im  0  0  0  0  0
        0  0  0  -1im  0  0  0  0
        1im  0  0  0  0  0  0  0
        0  1im  0  0  0  0  0  0
        0  0  0  0  0  0  -1im  0
        0  0  0  0  0  0  0  -1im
        0  0  0  0  1im  0  0  0
        0  0  0  0  0  1im  0  0
    ]

    σ̂_z_z_x= ComplexF64[
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
           ]
    σ̂_y_I_I= ComplexF64[
        0  0  0  0  -1im  0  0  0
        0  0  0  0  0  -1im  0  0
        0  0  0  0  0  0  -1im  0
        0  0  0  0  0  0  0  -1im
        1im  0  0  0  0  0  0  0
        0  1im  0  0  0  0  0  0
        0  0  1im  0  0  0  0  0
        0  0  0  1im  0  0  0  0
           ]

8×8 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0-1.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0-1.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0-1.0im
 0.0+1.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+1.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+1.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im

In [ ]:
J1 = 0.047
J2 = 0.097
J3 = 0.028

0.028

In [ ]:
Ω1 = 1
Ω2 = 3
Ω3 = 2

2

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω1=Ω1, ϵ1=ϵ1,ϵ2=ϵ2,ϵ3=ϵ3,ϵ4=ϵ4,ϵ5=ϵ5, Ω2=Ω2, Ω3=Ω3, J1=J1, J2=J2, J3=J3)
    Iσ̂_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0  1  0
        0  0  0  -1
    ]
    Iσ̂_x = ComplexF64[
        0  1  0  0
        1  0  0  0
        0  0  0  1
        0  0  1  0
    ]
    σ̂_zI = ComplexF64[
        1  0  0  0
        0  1  0  0
        0  0 -1  0
        0  0  0  -1
    ]
    σ̂_x_x = ComplexF64[
        0  0  0  1
        0  0  1  0
        0  1  0  0
        1  0  0  0
    ]
    σ̂_z_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0 -1  0
        0  0  0  1
    ]
    σ̂_y_y = ComplexF64[
        0  0  0 -1
        0  0  1  0
        0  1  0  0
       -1  0  0  0
    ]
    σ̂_z_z_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0 -1
    ]
    I_σ̂_z_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]
    σ̂_z_I_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]
     σ̂_z_I_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
    ]
    I_I_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0 -1
    ]
    I_σ̂_z_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
    ]
    I_I_σ̂_x = ComplexF64[
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
    ]
    I_σ̂_xI = ComplexF64[
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
    ]
    σ̂_xI_I = ComplexF64[
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
    ]

    σ̂_x_σ̂_x_I = ComplexF64[
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
    ]
    σ̂_y_σ̂_y_I = ComplexF64[
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0 -1
        0  0  0  0  1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  1  0  0  0  0  0
        0  0  0  1  0  0  0  0
       -1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
    ]
    σ̂_z_σ̂_z_I = ComplexF64[
        1  0  0  0  0  0  0  0
        0  1  0  0  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0  0  0  1  0
        0  0  0  0  0  0  0  1
    ]
    σ̂_xI_σ̂_x = ComplexF64[
        0  0  0  0  0  1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0  1  0  0  0  0  0
    ]
    σ̂_yI_σ̂_y = ComplexF64[
        0  0  0  0  0 -1  0  0
        0  0  0  0  1  0  0  0
        0  0  0  0  0  0  0 -1
        0  0  0  0  0  0  1  0
        0  1  0  0  0  0  0  0
       -1  0  0  0  0  0  0  0
        0  0  0  1  0  0  0  0
        0  0 -1  0  0  0  0  0
    ]
    σ̂_zI_σ̂_z = ComplexF64[
        1  0  0  0  0  0  0  0
        0 -1  0  0  0  0  0  0
        0  0  1  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  1  0  0
        0  0  0  0  0  0 -1  0
        0  0  0  0  0  0  0  1
    ]

    I_I_σ̂_y = ComplexF64[
        0  -1im  0  0  0  0  0  0
        1im  0  0  0  0  0  0  0
        0  0  0  -1im  0  0  0  0
        0  0  1im  0  0  0  0  0
        0  0  0  0  0  -1im  0  0
        0  0  0  0  1im  0  0  0
        0  0  0  0  0  0  0  -1im
        0  0  0  0  0  0  1im  0
    ]
    I_σ̂_y_I = ComplexF64[
        0  0  -1im  0  0  0  0  0
        0  0  0  -1im  0  0  0  0
        1im  0  0  0  0  0  0  0
        0  1im  0  0  0  0  0  0
        0  0  0  0  0  0  -1im  0
        0  0  0  0  0  0  0  -1im
        0  0  0  0  1im  0  0  0
        0  0  0  0  0  1im  0  0
    ]

    σ̂_z_z_x= ComplexF64[
        0  1  0  0  0  0  0  0
        1  0  0  0  0  0  0  0
        0  0  0 -1  0  0  0  0
        0  0 -1  0  0  0  0  0
        0  0  0  0  0 -1  0  0
        0  0  0  0 -1  0  0  0
        0  0  0  0  0  0  0  1
        0  0  0  0  0  0  1  0
           ]
    σ̂_y_I_I= ComplexF64[
        0  0  0  0  -1im  0  0  0
        0  0  0  0  0  -1im  0  0
        0  0  0  0  0  0  -1im  0
        0  0  0  0  0  0  0  -1im
        1im  0  0  0  0  0  0  0
        0  1im  0  0  0  0  0  0
        0  0  1im  0  0  0  0  0
        0  0  0  1im  0  0  0  0
           ]
    Ĥ₀ = -0.5 * (Ω1 * σ̂_z_I_I + Ω2 *I_σ̂_z_I + Ω3 * I_I_σ̂_z)  + J1 * (I_σ̂_z_σ̂_z) +J2 *(σ̂_z_I_σ̂_z) + J3 * ( σ̂_z_σ̂_z_I)
    Ĥ₁ = I_I_σ̂_x
    Ĥ2 = I_σ̂_xI
    Ĥ3 = σ̂_xI_I
    Ĥ4 = I_I_σ̂_y
    Ĥ5 = I_σ̂_y_I
    Ĥ6 = σ̂_y_I_I
    return (Ĥ₀, (Ĥ₁, ϵ1), (Ĥ2, ϵ2), (Ĥ3, ϵ3), (Ĥ4, ϵ4), (Ĥ5, ϵ5), (Ĥ6, ϵ6))
end;


In [ ]:
H = hamiltonian();
H

(ComplexF64[-2.828 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.1159999999999999 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.884 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 3.172 + 0.0im], (ComplexF64[0.0 + 0.0im 1.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 1.0 + 0.0im 0.0 + 0.0im], ϵ1), (ComplexF64[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], ϵ2), (ComplexF64[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], ϵ3), (ComplexF64[0.0 + 0.0im 0.0 - 1.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 1.0im 0.0

In [ ]:
tlist = collect(range(0, 700, length=7000));

In [ ]:
Pkg.add("Plots")
using Plots
Plots.default(
    linewidth               = 3,
    size                    = (700, 450),
    legend                  = :right,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [91a5bcdd] + Plots v1.38.15
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function ket(label)
    result = Dict("000" => Vector{ComplexF64}([1, 0, 0, 0, 0, 0, 0, 0]), "001" => Vector{ComplexF64}([0, 1, 0, 0, 0, 0, 0, 0]), "010" => Vector{ComplexF64}([0, 0, 1, 0, 0, 0, 0, 0]), "011" => Vector{ComplexF64}([0, 0, 0, 1, 0, 0, 0, 0])
    , "100" => Vector{ComplexF64}([0, 0, 0, 0, 1, 0, 0, 0]), "101" => Vector{ComplexF64}([0, 0, 0, 0, 0, 1, 0, 0]), "110" => Vector{ComplexF64}([0, 0, 0, 0, 0, 0, 1, 0]), "111" => Vector{ComplexF64}([0, 0, 0, 0, 0, 0, 0, 1]))
    return result[string(label)]
end;

In [ ]:
basis = [ket("000"),ket("001"),ket("010"),ket("011"),ket("100"),ket("101"),ket("110"),ket("111")];

In [ ]:
using LinearAlgebra

In [ ]:
basis_tgt=Array[]
for i in 1:100
   b= transpose(RandU_3Q[i]) * basis
   push!(basis_tgt, b)

end

In [ ]:
objectives=Array[]
for i in 1:100
   obj = [Objective(initial_state = Ψ, target_state = Ψtgt, generator = H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt[i])];
    push!(objectives, obj)
end

In [ ]:
problem=[]
for i in 1:100

    prob = ControlProblem(
    objectives=objectives[i],
    tlist=tlist,
    pulse_options=Dict(),
    iter_stop=800,
    J_T=QuantumControl.Functionals.J_T_sm,
    check_convergence=res -> begin
        ((res.J_T < 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    );
    push!(problem, prob)
end

In [ ]:
Pkg.add("QuantumPropagators")
using QuantumPropagators

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [7bf12567] + QuantumPropagators v0.6.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function store_pulses(wrk, iteration, _...)
    L = length(wrk.controls)
    ϵ_opt = reshape(wrk.pulsevals, L, :)
    opt_pulses = [QuantumControl.Controls.discretize_on_midpoints(ϵ_opt[l, :], tlist) for l=1:L]
    return Tuple(opt_pulses)
end

store_pulses (generic function with 1 method)

In [ ]:
Pkg.add("GRAPE")
using GRAPE

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6b52fcaf] + GRAPE v0.5.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
wrk=[]
for i in 1:100
  wrkk=GRAPE.GrapeWrk(problem[i]; )
  push!(wrk, wrkk)
end


In [ ]:
using QuantumControlBase

In [ ]:
results1=[]

for i in 1:5
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results1, opt_result_LBFGSB)
end


 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.59e-01   4.11e-01        n/a    1(0)    16.4
     1   8.31e-01   1.17e+00  -1.28e-01    3(0)    23.2
     2   8.02e-01   1.56e+00  -2.87e-02    2(0)    14.7
     3   6.32e-01   1.96e+00  -1.70e-01    1(0)     7.6
     4   5.17e-01   2.16e+00  -1.16e-01    2(0)    14.7
     5   3.85e-01   1.91e+00  -1.32e-01    1(0)     7.6
     6   2.68e-01   1.28e+00  -1.17e-01    1(0)     8.0
     7   2.23e-01   1.48e+00  -4.46e-02    1(0)     7.0
     8   1.90e-01   1.35e+00  -3.39e-02    1(0)     7.8
     9   1.48e-01   6.84e-01  -4.16e-02    1(0)     7.0
    10   1.27e-01   5.23e-01  -2.10e-02    1(0)     7.5
    11   9.13e-02   9.48e-01  -3.56e-02    1(0)     7.9
    12   5.93e-02   7.15e-01  -3.21e-02    1(0)     7.1
    13   3.60e-02   6.74e-01  -2.33e-02    1(0)     7.6
    14   2.56e-02   4.20e-01  -1.04e-02    1(0)     7.8
    15   1.92e-02   3.28e-01  -6.44e-03    1(0)     7.4
    16   1.51e-02   2.49e-01  -4.01e-03    1(0) 

In [ ]:
J_F_t_3Q_J2_6G1=[]
for i in 1:5
  J_F=results1[i].J_T
  push!(J_F_t_3Q_J2_6G1, J_F)
end

In [ ]:
results2=[]

for i in 6:10
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results2, opt_result_LBFGSB)
end


 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.63e-01   4.03e-01        n/a    1(0)     7.8
     1   8.65e-01   1.20e+00  -9.80e-02    1(0)     7.1
     2   7.48e-01   1.21e+00  -1.16e-01    1(0)     7.8
     3   6.65e-01   1.84e+00  -8.34e-02    2(0)    15.4
     4   5.81e-01   2.09e+00  -8.39e-02    1(0)     7.7
     5   3.92e-01   2.03e+00  -1.89e-01    2(0)    14.8
     6   2.72e-01   2.10e+00  -1.19e-01    1(0)     7.7
     7   1.83e-01   1.62e+00  -8.94e-02    2(0)    14.8
     8   1.35e-01   1.45e+00  -4.77e-02    1(0)     7.7
     9   8.38e-02   7.17e-01  -5.15e-02    1(0)     7.2
    10   5.09e-02   6.58e-01  -3.29e-02    1(0)     7.5
    11   2.54e-02   5.91e-01  -2.55e-02    1(0)     7.9
    12   1.02e-02   4.27e-01  -1.52e-02    1(0)     7.0
    13   3.99e-03   2.44e-01  -6.21e-03    1(0)     7.7
    14   2.09e-03   1.42e-01  -1.89e-03    1(0)     7.9
    15   6.13e-04   7.54e-02  -1.48e-03    1(0)     7.1
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F) 

In [ ]:
J_F_t_3Q_J2_6G2=[]
for i in 1:5
  J_F=results2[i].J_T
  push!(J_F_t_3Q_J2_6G2, J_F)
end